# DS-SF-34 | 07 | Linear Regression | Codealong | Starter Code

In [1]:
import os

import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import scipy.stats as stats

In [2]:
def read_dataset():
    return pd.read_csv(os.path.join('..', 'datasets', 'dataset-07-zillow.csv'), index_col = 'ID')

df = read_dataset()

In [3]:
# TODO
df.head()

,Address,DateOfSale,SalePrice,IsAStudio,Beds,Baths,Size,LotSize,BuiltInYear
ID,,,,,,,,,
15063471,"55 Vandewater St APT 9, San Francisco, CA",12/4/15,0.71,0,1,NaN,550,NaN,1980
15063505,"740 Francisco St, San Francisco, CA",11/30/15,2.15,0,NaN,2.0,1430,2435,1948
15063609,"819 Francisco St, San Francisco, CA",11/12/15,5.60,0,2,3.5,2040,3920,1976
15064044,"199 Chestnut St APT 5, San Francisco, CA",12/11/15,1.50,0,1,1.0,1060,NaN,1930
15064257,"111 Chestnut St APT 403, San Francisco, CA",1/15/16,0.97,0,2,2.0,1299,NaN,1993


## Scale `Size` and `LotSize` from sqft to '1,000 sqft'

In [4]:
def scale_variables(df):
    df.Size /= 10 ** 3 # Size in 1,000 sqft
    df.LotSize /= 10 ** 3 # Lot size in 1,000 sqft

scale_variables(df)

In [5]:
df[ ['Size', 'LotSize'] ]

,Size,LotSize
ID,,
15063471,0.550,NaN
15063505,1.430,2.435
15063609,2.040,3.920
15064044,1.060,NaN
15064257,1.299,NaN
...,...,...
2124214951,0.264,NaN
2126960082,0.691,NaN
2128308939,1.738,2.299


## Part A | Linear Regression with _statsmodels_' `OLS`

- (http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.OLS.html)

### `SalePrice` as a function of `Size`

In [11]:
def Xy(df):
    # TODO: X has to be a DataFrame 
    # TODO: y has to be a Series
    X = df[['Size']]
    y = df.SalePrice

    return X, y

X, y = Xy(df)  # running the funstion to define X and y

model = smf.OLS(y, X).fit()

model.summary()

# running the model with NaN/missing values will break the model.. you will get back NaNs instead of useful info

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 08 May 2017   Prob (F-statistic):                nan
Time:                        19:30:00   Log-Likelihood:                    nan
No. Observations:                1000   AIC:                               nan
Df Residuals:                    1000   BIC:                               nan
Df Model:                          -1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Size              nan        nan        nan        nan           nan       nan
==============================================================================
Omnibus:                          nan   Durbin-Watson:                     nan
Prob(Omnibus):                    nan   Jarque-Bera (JB):                  nan
Skew:                             nan   Prob(JB):                          nan
Kurtosis:                         nan   Cond. No.                          nan
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### `SalePrice` as a function of `Size` - Take 2

In [22]:
def Xy_2(df):    # another benefit of creating a function is you can edit the dataframe 
    # TODO: X    # and call it df and it won't change your original dataset
    # TODO: y
    df = df.dropna(subset=['Size'],axis=0)  # drop only the NULL values for the feature of interest
    X = df[['Size']]
    y = df.SalePrice
    
    return X, y

X, y = Xy_2(df)

model = smf.OLS(y, X).fit()

model.summary()

#without an intercept you have a feature matrix of 1 column and there is no intercept in your model output... :(

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.565
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     1255.
Date:                Mon, 08 May 2017   Prob (F-statistic):          7.83e-177
Time:                        19:36:33   Log-Likelihood:                -1689.6
No. Observations:                 967   AIC:                             3381.
Df Residuals:                     966   BIC:                             3386.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Size           0.8176      0.023     35.426      0.000         0.772     0.863
==============================================================================
Omnibus:                     1830.896   Durbin-Watson:                   1.722
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3370566.094
Skew:                          13.300   Prob(JB):                         0.00
Kurtosis:                     291.005   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### `SalePrice` as a function of `Size` - Take 3

- (http://statsmodels.sourceforge.net/devel/generated/statsmodels.tools.tools.add_constant.html)

In [23]:
def Xy_3(df):
    # TODO: X
    # TODO: y
    df = df.dropna(subset=['Size'],axis=0)  # drop only the NULL values for the feature of interest
    X = sm.tools.tools.add_constant(df[['Size']]) # adds a constant so the model will give you a slope and an intercept 
    y = df.SalePrice

    return X, y

X, y = Xy_3(df)

model = smf.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     297.4
Date:                Mon, 08 May 2017   Prob (F-statistic):           2.67e-58
Time:                        19:42:19   Log-Likelihood:                -1687.9
No. Observations:                 967   AIC:                             3380.
Df Residuals:                     965   BIC:                             3390.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.1551      0.084      1.842      0.066        -0.010     0.320
Size           0.7497      0.043     17.246      0.000         0.664     0.835
==============================================================================
Omnibus:                     1842.865   Durbin-Watson:                   1.704
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3398350.943
Skew:                          13.502   Prob(JB):                         0.00
Kurtosis:                     292.162   Cond. No.                         4.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# SalePrice = 0.1551 + 0.7497 * Size <- linear equation
# Null Hypothesis here is that B0 = 0 (assume the model is trash - we want to disprove this)
# If beta1 is 0, there is no relationship
# because we want to know if the variable contributes to the model
# the coefficient is significant

### Making predictions

In [24]:
predict_X = pd.DataFrame({'Size': [1.2, 1.4, 1.6]}, columns = ['Size'])
predict_X = sm.add_constant(predict_X)

In [25]:
predict_X

,const,Size
0,1,1.2
1,1,1.4
2,1,1.6


In [26]:
predict_y = model.predict(predict_X)

In [27]:
predict_y

array([ 1.05472548,  1.2046711 ,  1.35461672])

In [28]:
type(predict_y)

numpy.ndarray

### Model's parameters

In [31]:
# TODO

### t-values

In [ ]:
# TODO

### p-values

In [ ]:
# TODO

### Confidence Intervals

In [ ]:
# TODO

## Part B | The 68 - 90 - 95 - 99.7 Rule

- (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)

In [35]:
stats.norm.cdf(2) - stats.norm.cdf(-2)
# TODO

0.95449973610364158

In [36]:
print "For normally distributed data:"
for z in [1, 1.65, 2, 3]:
    print "\t- {:3.2f}% of it is between +/- {:1.2f} sigma(s)".\
        format((stats.norm.cdf(z) - stats.norm.cdf(-z)) * 100, z)

For normally distributed data:
	- 68.27% of it is between +/- 1.00 sigma(s)
	- 90.11% of it is between +/- 1.65 sigma(s)
	- 95.45% of it is between +/- 2.00 sigma(s)
	- 99.73% of it is between +/- 3.00 sigma(s)


> ### `norm.ppf` (percent point function) is the  inverse of `norm.cdf`:

In [37]:
stats.norm.ppf(stats.norm.cdf(1))

1.0

> ### $\sigma$ for the 90% rule?

In [ ]:
# TODO

## Part C | Linear Regression with _statsmodels_' `ols`

- (http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/formulas.html)

In [51]:
# TODO - lower case ols is different the upper case OLS
model = smf.ols(formula = 'SalePrice ~ Size', data=df).fit() #automatically adds the intercept  -1 to remove
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     297.4
Date:                Mon, 08 May 2017   Prob (F-statistic):           2.67e-58
Time:                        21:18:32   Log-Likelihood:                -1687.9
No. Observations:                 967   AIC:                             3380.
Df Residuals:                     965   BIC:                             3390.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.1551      0.084      1.842      0.066        -0.010     0.320
Size           0.7497      0.043     17.246      0.000         0.664     0.835
==============================================================================
Omnibus:                     1842.865   Durbin-Watson:                   1.704
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3398350.943
Skew:                          13.502   Prob(JB):                         0.00
Kurtosis:                     292.162   Cond. No.                         4.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### `SalePrice` as a function of `Size` without `Intercept`

In [52]:
# TODO
model = smf.ols(formula = 'SalePrice ~ Size - 1', data=df).fit() #automatically adds the intercept  -1 to remove
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.565
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     1255.
Date:                Mon, 08 May 2017   Prob (F-statistic):          7.83e-177
Time:                        21:18:55   Log-Likelihood:                -1689.6
No. Observations:                 967   AIC:                             3381.
Df Residuals:                     966   BIC:                             3386.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Size           0.8176      0.023     35.426      0.000         0.772     0.863
==============================================================================
Omnibus:                     1830.896   Durbin-Watson:                   1.722
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3370566.094
Skew:                          13.300   Prob(JB):                         0.00
Kurtosis:                     291.005   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Dropping outliers

In [55]:
def drop_outliers(df):
    print 'Dropping outliers'
    print '- n (before) =', df.shape[0]

    # TODO
    Q1 = df.Size.quantile(.25)
    Q2 = df.Size.quantile(.5)
    Q3 = df.Size.quantile(.75)

    print '- Q1         =', Q1, '($M)'
    print '- Q2/Median  =', Q2, '($M)'
    print '- Q3         =', Q3, '($M)'

    # TODO
    df = df.drop(df[])

    print '- n (after)  =', df.shape[0]

drop_outliers(df)

 Dropping outliers
- n (before) = 1000
- Q1         = 1.0275 ($M)
- Q2/Median  = 1.35 ($M)
- Q3         = 1.9475 ($M)
- n (after)  = 1000


### `SalePrice` as a function of `Size` (again)

In [ ]:
# TODO

model.summary()

## Part D | Checking modeling assumptions

In [ ]:
model = smf.ols(formula = 'SalePrice ~ Size', data = df).fit()

### Are the residuals normally distributed?

In [ ]:
# TODO

### Are the residuals normally distributed?  `.qqplot()`

In [ ]:
# TODO

### Are x and $\varepsilon$ independent?  `.plot_regress_exog()`

In [ ]:
# TODO

## Part E | Model's Fit and $R^2$

In [ ]:
df = read_dataset() # reload the dataset to get our outliers back...

scale_variables(df) # rescale the variables (use the function defined above)

### $SalePrice = \beta_0 + \beta_1 \times Size$

In [ ]:
X, y = Xy_3(df)

model = smf.OLS(y, X).fit()

model.summary()

In [ ]:
model.rsquared

In [ ]:
y_hat = model.predict(X)

var_y_hat = sum((y - y_hat) ** 2)
var_y = sum((y - y.mean()) ** 2)

1 - var_y_hat / var_y

### $SalePrice = \beta_1 \times Size$

In [ ]:
X, y = Xy_2(df)

model = smf.OLS(y, X).fit()

model.rsquared

> #### Is it real?

In [ ]:
y_hat = model.predict(X)

In [ ]:
1 - sum((y - y_hat) ** 2) / sum((y - y.mean()) ** 2)

## Part F | Calculating the t-value, p-value, and confidence interval for `Intercept`

- (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)

### $SalePrice = \beta_0 + \beta_1 \times Size$

In [ ]:
X, y = Xy_3(df)

model = smf.OLS(y, X).fit()

model.summary()

> ### Given the coefficient $\beta_0$ and the standard error $SE_{\beta_0}$

In [ ]:
print model.params.const
print model.bse.const

> ### $t\text{-}value_{\beta_0}$:

In [ ]:
model.tvalues.const

In [ ]:
# TODO

> ### $p\text{-}value_{\beta_0}$:

In [ ]:
model.pvalues.const

In [ ]:
print 'Degrees of freedom (df):', model.df_resid

In [ ]:
# TODO

> ### $CI_{\beta_0}$:

In [ ]:
model.conf_int().T.const

In [ ]:
# TODO

> ### (We can also calculate $SE_{\beta_0}$:)

In [ ]:
model.bse.const

In [ ]:
XTX_1 = np.linalg.inv(np.dot(X.T, X))

XTX_1

In [ ]:
v0 = XTX_1[0, 0]

v0

In [ ]:
sigma_hat = np.sqrt(1. / model.df_resid * (y ** 2 - y_hat ** 2).sum())

In [ ]:
np.sqrt(v0) * sigma_hat